<a href="https://colab.research.google.com/github/redman157/phamson/blob/master/Logistic_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pson2809","key":"bbe391f4594a3869ed38aaf983ee3043"}'}

In [2]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 64 Nov 29 00:39 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# List available datasets.
!kaggle datasets list

ref                                                           title                                                size  lastUpdated          downloadCount  
------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
kaggle/kaggle-survey-2018                                     2018 Kaggle ML & DS Survey Challenge                  4MB  2018-11-03 22:35:07           5046  
jacobbaruch/nba-player-of-the-week                            NBA player of the week                               16KB  2018-11-25 10:38:27           4335  
mehdidag/black-friday                                         Black Friday                                          5MB  2018-07-25 20:49:48          12530  
lava18/google-play-store-apps                                 Google Play Store Apps                                2MB  2018-09-18 20:49:49          23422  
center-for-policing-equity/data-science-for-good    

In [6]:
# Copy the stackoverflow data set locally.
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 33.1MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 28.4MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.80MB/s]


In [67]:
import tensorflow as tf
from tensorflow import keras 
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

training = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
gender =  pd.read_csv('gender_submission.csv')

X = training
y = training.Survived

feature_train = X.drop(['Name','PassengerId'],axis = 1)
gioitinh= {'male':0,'female':1}
feature_train.Sex.replace(gioitinh,inplace = True)

embarked = {'S':1,'C':2,'Q':3}
feature_train.Embarked.replace(embarked,inplace = True)
feature_train.Embarked.fillna(0,inplace = True)

feature_train['Family'] = feature_train.SibSp  + feature_train.Parch
feature_train.Family.loc[feature_train.Family > 0] =1 
feature_train.Family.loc[feature_train.Family == 0] = 0 

feature_train.Age.fillna(0,inplace = True)

feature_train.Cabin.fillna("X",inplace = True)
feature_train.Cabin = [i[0] for i in feature_train.Cabin]
s = feature_train.Cabin.sort_values().unique()
label = pd.factorize(s)

feature_train.Cabin = feature_train.Cabin.replace(label[1],label[0])
X = feature_train.drop(['Ticket','SibSp','Parch','Survived'],axis= 1)

X_train,X_val,y_train,y_val = train_test_split(X,y)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [68]:
test.head()
X_test = test
feature_test = X_test.drop(['Name','PassengerId'],axis = 1)

gioitinh= {'male':0,'female':1}
feature_test.Sex.replace(gioitinh,inplace = True)

embarked = {'S':0,'C':1,'Q':2}
feature_test.Embarked.replace(embarked,inplace= True)


feature_test['Family'] = feature_test.SibSp  + feature_train.Parch
feature_test.Family.loc[feature_test.Family > 0] =1 
feature_test.Family.loc[feature_test.Family == 0] = 0 

feature_test.Age.fillna(0,inplace = True)

feature_test.Cabin.fillna('X',inplace = True)
feature_test.Cabin = [i[0] for i in feature_test.Cabin]
h = feature_test.Cabin.sort_values().unique()
label = pd.factorize(h)
feature_test.Cabin = feature_test.Cabin.replace(label[1],label[0])

feature_test.Fare.fillna(0,inplace = True)
feature_test = feature_test.drop(['SibSp','Parch','Ticket'],axis = 1 )
X_test = feature_test
y_test = gender.Survived

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [69]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_validate
print(X_train.shape)
print(y_train.shape)

model = LogisticRegression()
model.fit(X_train,y_train)
pred = model.predict(X_test)
loss = mean_squared_error(y_test,pred)
print('Mean Squared Error: ', loss)
score = r2_score(y_test,pred)
print('Variabel accuracy: %.2f',score)
vals = cross_validate(model,X_val,y_val,)
print('val_score', vals)
confu = confusion_matrix(pred,y_test)
print('confu', confu)

(668, 7)
(668,)
Mean Squared Error:  0.0861244019138756
Variabel accuracy: %.2f 0.6278195488721805
val_score {'fit_time': array([0.00203586, 0.00169539, 0.00167227]), 'score_time': array([0.00049114, 0.00046253, 0.00050235]), 'test_score': array([0.81333333, 0.77027027, 0.81081081]), 'train_score': array([0.79054054, 0.81208054, 0.77852349])}
confu [[259  29]
 [  7 123]]


In [0]:
X_train = X_train.values
y_train = y_train.values
X_test = X_test.values
y_test = y_test.values

In [0]:
A = np.dot(X_train.T,X_train)
b = np.dot(X_train.T,y_train)
w = np.dot(np.linalg.pinv(A),b)



In [0]:
def sigmoid(X,derv = False):
  if(derv == True):
    return X * (1+X)
  return 1/(1 + np.exp(-X))
pred = sigmoid(np.dot(X_test,w))
def cost_function(X,y):
  return (-y* np.log(pred - ( y )* np.log(1-pred(X,w))))
def update_w(lr,epochs):
  batch_size = X.shape[1]
  w = np.matrix([0.0 for i in range(batch_size)])
  for i in range(epochs):
    error = np.repeat(pred - y.T,batch_size,axis = 0)
    error_derv = np.sum(np.matmul(error.T * X),axis = 1 )
    w = w - (lr/X_train.shape[0]) * error_derv
  return w

# update_w(0.01,100)
# def GD(X,y,lr):
#   epochs = X.shape[0]
#   batch_size = X.shape[1]
#   cost = [0.0 for i in range()]
#   for i in range(epochs):
#     cost = 

# fit_w(X_train,y_train,0.01,100) 
# error_dev = mse(pred - y_train)
  

In [59]:
h

array([ True,  True,  True, False,  True, False,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False,  True,  True, False,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False, False,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True,  True,  True, False,  True, False,  True,  True,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
       False,  True,